In [ ]:
import psi4
import numpy as np

## Part A: Initial setup and energy calculation

Below are (unoptimized) coordinates for nitrobenzene and a quick energy calculation using the method/basis set combination of B3LYP/3-21G.

In [ ]:
psi4.set_memory('2 GB')
psi4.set_num_threads(2)

nitrobenz = psi4.geometry("""
symmetry c1
0 1
C       -3.5048225421    0.0711805817    0.1456897967
C       -2.1242069042    0.0676957680    0.1437250554
C       -1.4565144627    1.2657898054    0.0112805274
C       -2.1243502782    2.4616659201   -0.1394727314
C       -3.5049153121    2.4578370923   -0.1457245349
C       -4.1936081427    1.2645153194    0.0001955136
H       -4.0381801262   -0.8505059514    0.2559173303
H       -1.5620288767   -0.8346363876    0.2389155097
H       -1.5619534389    3.3630228735   -0.2428628637
H       -4.0382012347    3.3785626398   -0.2639829256
H       -5.2650389640    1.2641688843   -0.0022762561
N       -0.0085078655    1.2648596634   -0.0056641832
O        0.5639468379    0.1670702678   -0.1297708787
O        0.5668300231    2.3598431617    0.1306822195
""")

psi4.set_output_file('00_initial_energy.dat', False)
# Uncomment this line and change charge / multiplicity above to perform calculations on the anion.
# If you already carried out calculations on the neutral molecule in this notebook,
# don't forget to click Kernel -> Restart & Clear Output before working on the anion.
# psi4.set_options({'reference':'UKS'})
psi4.energy('B3LYP/3-21G')

## Part B: Geometry optimization (gas phase)

In [ ]:
psi4.set_output_file('01_geometry_optimization.dat', False)
psi4.set_options({'g_convergence': 'gau_tight'}) # this forces the optimizer to get close to the minimum
psi4.optimize('B3LYP/3-21G', molecule=nitrobenz)

## Part C: Frequency calculation

In [ ]:
psi4.set_output_file('02_frequency_analysis.dat', False)
b3lyp_321g_energy, b3lyp_321g_wfn = psi4.frequency('B3LYP/3-21G', molecule=nitrobenz, return_wfn=True, dertype='gradient')

In [ ]:
def parse_free_energy(filename):
    """ Parse out the free energy from a Psi4 vibrational analysis output file in a.u. """
    for line in open(filename).readlines():
        if "Correction G" in line:
            return float(line.split()[-2])

# Below this space, write a call to the function above and pass in the correct file name.



## Part D: Energy calculation with large basis set + PCM solvent model

In [ ]:
psi4.set_options({
  'pcm': True,
  'pcm_scf_type': 'total',
})

psi4.pcm_helper("""
   Units = Angstrom
   Medium {
   SolverType = CPCM
   Solvent = Acetonitrile
   }
   Cavity {
   RadiiSet = UFF
   Type = GePol
   Scaling = False
   Area = 0.3
   Mode = Implicit
   }
""")

psi4.set_output_file('03_solvent_energy.dat', False)
psi4.energy('B3LYP/heavy-aug-cc-pVDZ', molecule=nitrobenz, return_wfn=True)